# Obter dados no drive

In [ ]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)
from google.colab import drive
import pandas as pd
import random
import os
import time
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, Callback
from glob import glob
from sklearn.model_selection import train_test_split
import numpy as np
from keras import layers, models, Input, regularizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
drive.mount('/content/drive')

# Carregamento de dados - Sem Data Augmentation


In [ ]:
images = []
masks = []

for path in glob('/content/drive/Shared drives/Grupo T de Tech/Data/dataset_inteli/cropped_images/*/*'):
  images.append(path + '/image.tif')
  masks.append(path + '/mask.png')

In [ ]:
# Função para carregar e pré-processar uma imagem e sua máscara
def load_and_preprocess_image(image_path, mask_path, target_size):

    image = load_img(image_path, target_size=target_size)
    image = img_to_array(image) / 255.0  # Normalização entre 0 e 1

    mask = load_img(mask_path, target_size=target_size, color_mode='grayscale')
    mask = img_to_array(mask) / 255.0  # Normalização entre 0 e 1

    return image, mask

In [ ]:
with tf.device('/gpu:0'):
    # Lista para armazenar imagens e máscaras pré-processadas
    images_processed = []
    masks_processed = []

    # Carregar e pré-processar todas as imagens e máscaras
    for img_path, mask_path in zip(images, masks):
        img, mask = load_and_preprocess_image(img_path, mask_path, target_size=(256, 256))
        images_processed.append(img)
        masks_processed.append(mask)

    # Converter para arrays numpy
    images_processed = np.array(images_processed)
    masks_processed = np.array(masks_processed)


In [ ]:
images_processed

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(images_processed, masks_processed, test_size=0.3, random_state=42)

# Definição de parâmetros e modelo

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [ ]:
# Função para calcular a sigmoide e converter para 0 ou 1 o output
class ThresholdLayer(tf.keras.layers.Layer):
    def call(self, inputs):
        return tf.where(inputs < 0.5, 0.0, 1.0)

In [ ]:
# Métrica de acurácia customizada
def custom_accuracy(y_train, y_val):
    # Calcular a acurácia considerando uma tolerância de 0.5 na predição
    y_val_binary = tf.round(y_val)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(y_train, y_val_binary), tf.float32))
    return accuracy

In [ ]:
def Unet(input_shape, dropout_rate):
    num_filters=(16, 32, 64, 128)
    kernel_size=3
    val_reg=0.001
    inputs = tf.keras.Input(shape=input_shape)
    reg = regularizers.L2(val_reg)

    # Encoder (contraction path)
    conv1 = layers.Conv2D(num_filters[0], kernel_size, activation='relu', padding='same', kernel_regularizer=reg)(inputs)
    conv1 = layers.Conv2D(num_filters[0], kernel_size, activation='relu', padding='same', kernel_regularizer=reg)(conv1)
    drop1 = layers.Dropout(dropout_rate)(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(drop1)

    conv2 = layers.Conv2D(num_filters[1], kernel_size, activation='relu', padding='same', kernel_regularizer=reg)(pool1)
    conv2 = layers.Conv2D(num_filters[1], kernel_size, activation='relu', padding='same', kernel_regularizer=reg)(conv2)
    drop2 = layers.Dropout(dropout_rate)(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(drop2)

    conv3 = layers.Conv2D(num_filters[2], kernel_size, activation='relu', padding='same', kernel_regularizer=reg)(pool2)
    conv3 = layers.Conv2D(num_filters[2], kernel_size, activation='relu', padding='same', kernel_regularizer=reg)(conv3)
    drop3 = layers.Dropout(dropout_rate)(conv3)
    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(drop3)

    # Bottleneck
    conv4 = layers.Conv2D(num_filters[3], kernel_size, activation='relu', padding='same', kernel_regularizer=reg)(pool3)
    conv4 = layers.Conv2D(num_filters[3], kernel_size, activation='relu', padding='same', kernel_regularizer=reg)(conv4)
    drop4 = layers.Dropout(dropout_rate)(conv4)

    # Decoder (expansion path)
    up5 = layers.Conv2DTranspose(num_filters[2], (2, 2), strides=(2, 2), padding='same')(drop4)
    merge5 = layers.concatenate([conv3, up5], axis=3)
    conv5 = layers.Conv2D(num_filters[2], kernel_size, activation='relu', padding='same', kernel_regularizer=reg)(merge5)
    conv5 = layers.Conv2D(num_filters[2], kernel_size, activation='relu', padding='same', kernel_regularizer=reg)(conv5)

    up6 = layers.Conv2DTranspose(num_filters[1], (2, 2), strides=(2, 2), padding='same')(conv5)
    merge6 = layers.concatenate([conv2, up6], axis=3)
    conv6 = layers.Conv2D(num_filters[1], kernel_size, activation='relu', padding='same', kernel_regularizer=reg)(merge6)
    conv6 = layers.Conv2D(num_filters[1], kernel_size, activation='relu', padding='same', kernel_regularizer=reg)(conv6)

    up7 = layers.Conv2DTranspose(num_filters[0], (2, 2), strides=(2, 2), padding='same')(conv6)
    merge7 = layers.concatenate([conv1, up7], axis=3)
    conv7 = layers.Conv2D(num_filters[0], kernel_size, activation='relu', padding='same', kernel_regularizer=reg)(merge7)
    conv7 = layers.Conv2D(num_filters[0], kernel_size, activation='relu', padding='same', kernel_regularizer=reg)(conv7)

    outputs = layers.Conv2D(1, 1, activation='sigmoid')(conv7)  # Saída com um canal (máscara binária)

    threshold_output = ThresholdLayer()(outputs)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
# Criar modelo U-Net
model = Unet(input_shape=(256, 256, 3), dropout_rate=0.3)
model.summary()

# Treino do modelo - GPU

In [ ]:
callbacks = early_stopping
with tf.device('/gpu:0'):
    # Compilar o modelo
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=[custom_accuracy])

    # Calcula o tempo de treino
    start_time = time.time()

    # Definições
    max_epochs = 200
    batch_size = 16

    # Treinar o modelo
    H = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=max_epochs, batch_size=batch_size, callbacks=callbacks)

    # Para o cronômetro e salva o tempo de treino
    training_time_gpu = time.time() - start_time

## Avaliação do modelo - GPU

In [ ]:
with tf.device('/gpu:0'):
    # Mostrando resultados
    plt.style.use("ggplot")
    plt.figure()
    plt.plot(H.epoch, H.history["loss"], label="train_loss")
    plt.plot(H.epoch, H.history["val_loss"], label="val_loss")
    plt.plot(H.epoch, H.history["custom_accuracy"], label="train_acc")
    plt.plot(H.epoch, H.history["val_custom_accuracy"], label="val_acc")
    plt.title("Training Loss and Accuracy - GPU")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend()
    plt.show()

In [ ]:
with tf.device('/gpu:0'):
    # Calcula o tempo de inferência
    start_time = time.time()

    # Avaliar o modelo nos dados de teste
    results = model.evaluate(X_val, y_val)

    # Para o cronômetro e salva o tempo de treino
    inference_time_gpu = time.time() - start_time

    print("Test Loss - GPU:", results[0])
    print("Test Accuracy - GPU:", results[1])

    # Prever máscaras usando o modelo
    predicted_masks = model.predict(X_val)

In [ ]:
with tf.device('/gpu:0'):
    # Obter métricas de precisão e perda do histórico de treinamento
    acc = H.history['custom_accuracy']
    val_acc = H.history['val_custom_accuracy']
    loss = H.history['loss']
    val_loss = H.history['val_loss']

    # Número de épocas
    epochs = range(1, len(acc) + 1)

    # Plotar precisão do conjunto de treino e validação
    plt.plot(epochs, acc, 'r', label='Precisão do Conjunto de Treino')
    plt.plot(epochs, val_acc, 'b', label='Precisão do Conjunto de Validação')
    plt.title('Precisão do Conjunto de Treino e Validação - GPU')
    plt.xlabel('Épocas')
    plt.ylabel('Precisão')
    plt.legend()
    plt.show()

    # Plotar perda do conjunto de treino e validação
    plt.plot(epochs, loss, 'r', label='Perda do Conjunto de Treino')
    plt.plot(epochs, val_loss, 'b', label='Perda do Conjunto de Validação')
    plt.title('Perda do Conjunto de Treino e Validação - GPU')
    plt.xlabel('Épocas')
    plt.ylabel('Perda')
    plt.legend()
    plt.show()

In [ ]:
with tf.device('/gpu:0'):
    # Gerar as saídas do modelo para um conjunto de entradas de teste
    saidas_modelo = model.predict(X_val)

    # Iterar sobre cada saída do modelo
    for i in range(len(X_val)):
        # Obter a entrada correspondente e a saída real
        img_entrada = X_val[i]
        img_saida_real = y_val[i]

        # Obter a saída gerada pelo modelo
        img_saida_modelo = saidas_modelo[i]

        # Mostrar as imagens
        plt.figure(figsize=(12, 4))

        plt.subplot(1, 3, 1)
        plt.imshow(img_entrada.squeeze(), cmap='gray')
        plt.title('Entrada')
        plt.axis('off')

        plt.subplot(1, 3, 2)
        plt.imshow(img_saida_real.squeeze(), cmap='gray')
        plt.title('Saída Esperada')
        plt.axis('off')

        plt.subplot(1, 3, 3)
        plt.imshow(img_saida_modelo.squeeze(), cmap='gray')
        plt.title('Saída do Modelo - GPU')
        plt.axis('off')

        plt.show()

In [ ]:
with tf.device('/gpu:0'):
    # Métricas do parceiro de Projeto:

    # Lista para armazenar os scores de IoU
    iou_scores = []
    # Calcular IoUs e determinar predições corretas
    correct_predictions = 0
    iou_threshold = 0.5
    for mask, result in zip(y_val, img_saida_modelo):
        intersection = np.logical_and(mask, result)
        union = np.logical_or(mask, result)
        iou_score = np.sum(intersection) / np.sum(union) if np.sum(union) != 0 else 0
        iou_scores.append(iou_score)
        # Verificar se a predição é considerada correta (IoU >= threshold)
        if iou_score >= iou_threshold:
            correct_predictions += 1
    # Calcular a média dos IoUs
    iou_mean = np.mean(iou_scores)
    print('Média dos IoU - GPU:', iou_mean)
    # Calcular Coverage Ratio (CovR)
    total_predictions = len(iou_scores)
    covr = correct_predictions / total_predictions if total_predictions > 0 else 0
    print('Coverage Ratio (CovR) - GPU:', covr)